In [2]:
import tensorflow as tf
from tensorflow import keras
import sys
sys.path.insert(0, 'C:/Users/richa/PycharmProjects/ai_mechatronics/')

print('TensorFlow version {}'.format(tf.__version__))
print('Keras version {}'.format(keras.__version__))


TensorFlow version 1.13.1
Keras version 2.2.4-tf


In [4]:
"""
Run this cell to convert the training/validation/test set to tfrecords

If you want to run this code, 
1. copy this cell to a .py file and add a line to change sys.path
2. change FLAGS.DEFAULT_DOWNLOAD to:
/data/cephfs/punim0811/Datasets/iNaturalist/
3. Change output_folder accordingly because I intentionally did not grant 
you writing permission to the above folder. 

Convertion time cost:
    train - 7800 seconds
    val - 100 seconds
    test - 1100 seconds
"""
from GeneralTools.misc_fun import FLAGS
FLAGS.DEFAULT_DOWNLOAD = '/media/richard/My Book/MyBackup/Data/Kaggle_iNaturalist_2019/'
from GeneralTools.inaturalist_func import images_to_tfrecords
import os.path
import json

key = 'train'  # choose from {'train', 'val', 'test'}
num_images_per_tfrecord = {'train': 11531, 'val': 3030, 'test': 17675}
target_size = 299  # change this if you want other image resolution
num_images_per_tfrecord = num_images_per_tfrecord[key]

# read json file
annotation_file = '{}2019.json'.format(key)
with open(os.path.join(FLAGS.DEFAULT_DOWNLOAD, annotation_file)) as data_file:
    image_annotations = json.load(data_file)

# extract image file names and classes if provided
images = image_annotations['images']
annotations = image_annotations['annotations'] if 'annotations' in image_annotations else None
image_names = [image['file_name'] for image in images]
image_class = None if annotations is None else [annotation['category_id'] for annotation in annotations]
image_index = 4
print('The {}-th validation image locates at {}; its class is {}'.format(
    image_index, image_names[image_index], 'unknown' if image_class is None else image_class[image_index]))
num_images = len(image_names)
print('There are {} images in {}'.format(num_images, annotation_file))

# configure folders to save the data
output_folder = os.path.join(
    FLAGS.DEFAULT_DOWNLOAD, 'tfrecords_{}/'.format(target_size))
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
output_filename = output_folder + key
print('The tfrecords are saved to {}'.format(output_filename))

# # uncomment the following lines to do the actual conversion
# images_to_tfrecords(
#     image_names, output_filename, num_images_per_tfrecord, 
#     image_class=image_class, target_size=299)


The 4-th validation image locates at train_val2019/Plants/739/ffa06f951e99de9d220aee2c3309b66c.jpg; its class is 739
There are 265213 images in train2019.json
The tfrecords are saved to /media/richard/My Book/MyBackup/Data/Kaggle_iNaturalist_2019/tfrecords_299/train


In [13]:
"""
This cell reads the train/vali/test tfrecords, and visualize one sample from
the batch.

If you want to run this code, 
1. copy this cell to a .py file and add a line to change sys.path
2. change FLAGS.DEFAULT_IN to:
/data/cephfs/punim0811/Datasets/iNaturalist/tfrecords_299/
or your local machine address accordingly if you decide to download 
some tfrecords file.
3. Note that on Spartan, im.show() would not work. However, you may save the 
example you want to visualize, e.g., im.save('test_image.jpg', 'JPEG')
"""
from GeneralTools.misc_fun import FLAGS
FLAGS.DEFAULT_IN = '/media/richard/ExtraStorage/Data/inaturalist_NHWC_800/'
FLAGS.IMAGE_FORMAT = 'channels_last'
FLAGS.IMAGE_FORMAT_ALIAS = 'NHWC'
from GeneralTools.inaturalist_func import ReadTFRecords
import os
import tensorflow as tf
from PIL import Image
import numpy as np

batch_size = 64
target_size = 800
key = 'train'
data_size = {'train': 265213, 'val': 3030, 'test': 35350}
data_label = {'train': 1, 'val': 1, 'test': 0}
num_images = data_size[key]
steps_per_epoch = num_images // batch_size
skip_count = num_images % batch_size
num_labels = data_label[key]
num_classes = 1010
    
filenames = os.listdir(FLAGS.DEFAULT_IN)
filenames = [filename.replace('.tfrecords', '') for filename in filenames if key in filename]
print(filenames)

dataset = ReadTFRecords(
    filenames, num_labels=num_labels, batch_size=batch_size, buffer_size=500,
    skip_count=skip_count, num_threads=8, decode_jpeg=True, 
    use_one_hot_label=True, num_classes=num_classes)
dataset.image_preprocessor(3, target_size, target_size)
data_batch = dataset.next_batch()

with tf.Session() as sess:
    if key == 'test':
        x = sess.run(data_batch['x'])
    else:
        x, y = sess.run([data_batch['x'], data_batch['y']])

# visualize one sample from the batch
x_im = x[0] * 255
im = Image.fromarray(x_im.astype(np.uint8), 'RGB')
im.show()

if key in {'train', 'val'}:
    print(y.shape)
    print(y.dtype)


['train_091', 'train_095', 'train_039', 'train_006', 'train_021', 'train_016', 'train_003', 'train_108', 'train_071', 'train_009', 'train_031', 'train_107', 'train_093', 'train_106', 'train_020', 'train_089', 'train_034', 'train_049', 'train_017', 'train_022', 'train_102', 'train_068', 'train_057', 'train_062', 'train_112', 'train_063', 'train_096', 'train_037', 'train_036', 'train_123', 'train_004', 'train_001', 'train_101', 'train_100', 'train_104', 'train_040', 'train_000', 'train_044', 'train_013', 'train_066', 'train_073', 'train_094', 'train_023', 'train_055', 'train_072', 'train_024', 'train_069', 'train_046', 'train_120', 'train_041', 'train_014', 'train_076', 'train_121', 'train_111', 'train_092', 'train_075', 'train_090', 'train_085', 'train_025', 'train_084', 'train_015', 'train_083', 'train_019', 'train_081', 'train_010', 'train_067', 'train_124', 'train_030', 'train_079', 'train_060', 'train_058', 'train_026', 'train_042', 'train_114', 'train_086', 'train_127', 'train_122'

In [1]:
"""
This cell train a pre-trained model with extra layers

"""
# keras pretrained inception v3 model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, applications
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras import backend as K 

# load the model
model = applications.InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(target_size, target_size, 3))
# Freeze some layers
for layer in model.layers[:-20]:
    layer.trainable = False
#Adding custom layers 
# x = model.output
# x = Flatten()(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
# predictions = Dense(num_classes, activation='linear')(x)
# mdl = Model(input = model.input, output = predictions)
mdl = Sequential([
    model, Flatten(), Dense(1024, activation='relu'), 
    Dense(num_classes, activation='linear')])

mdl.compile(
    tf.train.AdamOptimizer(learning_rate=0.001), 
    loss=tf.losses.softmax_cross_entropy, metrics=['accuracy'])
history = mdl.fit(
    dataset.dataset, epochs=1, callbacks=None, steps_per_epoch=20,
    validation_data=None, validation_steps=200, verbose=1)



NameError: name 'target_size' is not defined

In [10]:
from GeneralTools.misc_fun import FLAGS
FLAGS.DEFAULT_IN = '/media/richard/ExtraStorage/Data/inaturalist_NHWC_299/'
FLAGS.IMAGE_FORMAT = 'channels_last'
FLAGS.IMAGE_FORMAT_ALIAS = 'NHWC'
import os
if FLAGS.MIXED_PRECISION:
    os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
from GeneralTools.inaturalist_func import read_inaturalist
from PIL import Image
import numpy as np

allow_growth = False
target_size = 299
num_classes = 1010
key = 'train'

# read the dataset
dataset_tr, steps_per_tr_epoch = read_inaturalist(
    key, batch_size=2, target_size=target_size, do_augment=True)
data_batch = dataset_tr.next_batch()

with tf.Session() as sess:
    if key == 'test':
        x = sess.run(data_batch['x'])
        y = ['None']
    else:
        x, y = sess.run([data_batch['x'], data_batch['y']])

# visualize one sample from the batch
x_im = x[0] * 255
im = Image.fromarray(x_im.astype(np.uint8), 'RGB')
im.show()
if key != 'test':
    print(np.amax(y[0]))


The following tfrecords are read: ['train_006', 'train_021', 'train_016', 'train_003', 'train_009', 'train_020', 'train_017', 'train_022', 'train_004', 'train_001', 'train_000', 'train_013', 'train_014', 'train_015', 'train_019', 'train_010', 'train_011', 'train_002', 'train_007', 'train_018', 'train_012', 'train_005', 'train_008']
Number of 1 instances skipped.
The dataset repeats for infinite number of epochs
0.900099


In [8]:
from GeneralTools.misc_fun import FLAGS
FLAGS.DEFAULT_IN = 'C:/Users/richa/PycharmProjects/ai_mechatronics/Datasets/tfrecords_299'
FLAGS.IMAGE_FORMAT = 'channels_last'
FLAGS.IMAGE_FORMAT_ALIAS = 'NHWC'
import os
if FLAGS.MIXED_PRECISION:
    os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
from GeneralTools.inaturalist_func import read_inaturalist
from GeneralTools.inception_preprocessing import preprocess_image
from PIL import Image
import numpy as np

allow_growth = False
target_size = 299
num_classes = 1010
key = 'train'

# read the dataset
dataset_tr, steps_per_tr_epoch = read_inaturalist(
    key, batch_size=1, target_size=target_size, do_augment=False)
data_batch = dataset_tr.next_batch()

image = data_batch['x'][0]

image_aug = preprocess_image(
    image, height=target_size, 
    width=target_size, is_training=True if key == 'train' else False, fast_mode=False)

with tf.Session() as sess:
    x, xa = sess.run([image, image_aug])

# visualize one sample from the batch
print((np.amin(x), np.amax(x)))
print((np.amin(xa), np.amax(xa)))

x_im = x * 255.0
x_im2 = (xa + 1.0) * 127.5
stack_im = np.hstack((x_im.astype(np.uint8),x_im2.astype(np.uint8)))
Image.fromarray(stack_im).show()



The following tfrecords are read: ['train_000', 'train_001', 'train_002', 'train_003', 'train_004', 'train_005', 'train_006', 'train_007', 'train_008', 'train_009', 'train_010', 'train_011', 'train_012', 'train_013', 'train_014', 'train_015', 'train_016', 'train_017', 'train_018', 'train_019', 'train_020', 'train_021', 'train_022']
The dataset repeats for infinite number of epochs


(0.0, 1.0)
(-0.98921096, 0.7570392)
